# Amazon Data Cleaning project


This Jupyter notebook is part of an automation project for data cleaning in Amazon Ads. Managing two distinct ad accounts, each with its unique set of brands, campaign types, and targetings, poses significant challenges in data processing and management. Initially, the data cleaning process was cumbersome and inefficient, primarily conducted in Excel, which proved to be unreliable and time-consuming.

The primary objective of this project is to streamline the data cleaning process, reducing the steps involved to a minimum and significantly cutting down the time required. This notebook demonstrates a sample of the data cleaning procedure applied to various sheets, which previously took 1-2 hours, but can now be completed in a matter of seconds.

Along the way I've created multiple functions that will come in handy in my future projects!

Future enhancements include the integration of Watchdog, an automation tool, to further simplify the process. The goal is to achieve a system where new files are automatically detected and processed, producing the final output without manual intervention.

This code, along with its detailed documentation, will be shared on my GitHub account, offering insights into the methods and reasoning behind this automated data cleaning approach.


In [1]:
# Importing Libraries for the data cleaning process

import pandas as pd
import unicodedata #  The unicodedata module is imported to handle Arabic characters effectively.
import re #used to extract the number of the file
import sys
import os
import concurrent.futures # for more efficient execution of importing files.

In [8]:
# Path to the directory with your files
path_main_file = 'main file' # platform results was only available for 90 days, which is why I've created a main file that contains all the consolidated data to add to it  
Path_new_file = 'new_file_path' # path for the new file
products_file_path = 'Product_file' # products are coded, So I imported a file with the product name - code - brand for better analysis

def read_excel_file(file_path, columns=None):
    """This function is to make importing xlsx more robust"""
    if columns:
        return pd.read_excel(file_path, usecols=columns)
    else:
        return pd.read_excel(file_path)

with concurrent.futures.ThreadPoolExecutor() as executor:
    SPST_new_file,products_file_main = executor.map(read_excel_file,[Path_new_file,products_file_path])


SPST_main = path_main_file # I added a new variable so that I don't need to re-import the files again
SPASIN_main_file = pd.read_csv(path_main_file)
SPASIN_main_file['Date'] = pd.to_datetime(SPASIN_main_file['Date'])

SPST = SPST_new_file # I added a new variable so that I don't need to re-import the files again
SPST['source_file'] = os.path.basename(Path_new_file)
SPST['source_file_n'] = SPST['source_file'].apply(lambda x: re.search(r'\((\d+)\)', x).group(1) if re.search(r'\((\d+)\)', x) else None)
SPST['source_file_number'] = SPST['source_file_n'].astype(int)
SPST['Date'] = pd.to_datetime(SPST['Date'])

products_file = products_file_main # I added a new variable so that I don't need to re-import the files again

c:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
def Sponsored_Product_SPST(SPST,SPASIN_main_file):
    """This script processes and merges the newly imported data with the primary dataset."""
    # formula to get the type of targeting
    def determine_targeting(SPST):

        """We had 4 different targeting types, this function is designed to differentiate between them
            and it's specific campaign"""
        if SPST['Match Type'] == '-' and 'category' in SPST['Targeting']:
            return 'Category Targeting'
            
        elif SPST['Match Type'] == '-' and 'asin' in SPST['Targeting']:
            return 'ASIN Targeting'
        
        elif SPST['Match Type'] != '-':
            return 'Keyword Targeting'

        else:
            return 'Automatic Targeting'

    # formula to get the specific (Keyword - Category - ASIN) we're targeting
    def Keyword_category_asin(SPST):
            """Within each targeting type we needed to identify the specific targeting
                ex.the followng is an example for the report category="Powdered Spices & Seasonings"
                 we needed to export the category within (Powdered Spices & Seasonings)
                for ASINs it was the same thing: asin="B09V51WF73", this was needed to be able
                 to identify the ASIN and its brand for better understanding of performance. """
            try:
                if SPST['Match Type'] == '-':
                    start_pos = SPST['Targeting'].find('"')+1
                    end_pos = SPST['Targeting'].rfind('"')-1
                    if start_pos > 0 and end_pos > start_pos:
                        # Extract substring from one position after '=' to the end
                        return SPST['Targeting'][start_pos:end_pos]
                    else:
                        # if '=' is not found, return the full 'Targeting' string
                        return SPST['Targeting']
                else:
                    # if '-' isn't found, return full 'Targeting' string
                    return SPST['Targeting']
            
            except:
                # if there is an error return the 'Targeting' string
                return 'Automatic Targeting'

    # Adding columns with the clearly identified targeting and targeting types.
    SPST['Targeting Type'] = SPST.apply(determine_targeting, axis=1)
    SPST['Keyword_category_asin'] = SPST.apply(Keyword_category_asin, axis=1)


    # removing unicodedata and converting to float
    def clean_convert_to_numeric(column):
        """Because Arabic letters are RTL this posed an interesting challenge as even after removed it left a space that couldn't be deleted, therefore i needed to deal with the RTL characters and remove them"""

        def remove_rtl_characters(text):
            """Removing RTL characters to process the data correctly"""
            return ''.join(char for char in text if unicodedata.category(char) != 'Lm' and unicodedata.category(char) != 'Cf')
        
        cleaned_column = column.astype(str).apply(remove_rtl_characters)
        cleaned_column = cleaned_column.str.replace('ج.م.', '', regex=False)
        return pd.to_numeric(cleaned_column, errors='coerce')

    SPST['14 Day Total Sales'] = clean_convert_to_numeric(SPST['14 Day Total Sales'])
    SPST['Spend'] = clean_convert_to_numeric(SPST['Spend']) 
    products_file['ASIN'] = products_file['ASIN'].str.lower()
    

    # merging with the product file to get a better understanding of which ASINs we appear on
    SPST = SPST.merge(products_file,how='left',left_on='Customer Search Term', right_on='ASIN')
    SPST['Brand Description'].fillna('', inplace=True)
    
    # remove rows with 0 impressions
    cleaned_file = SPST[SPST['Impressions'] != 0]

    # Renaming columns 
    cleaned_file = cleaned_file.rename(columns={'14 Day Total Sales': 'Total Sales', '14 Day Total Orders (#)': 'Total Orders', '14 Day Total Units (#)': 'Total Units','14 Day Advertised ASIN Units (#)':'Total Units Advertised ASINS','14 Day Brand Halo ASIN Units (#)':'Total Units Brand Halo ASIN', '14 Day Advertised ASIN Sales':'Total Sales Advertised ASINS','14 Day Brand Halo ASIN Sales':'Total Sales Brand Halo ASIN'})
    
    # labeling branded and non branded based on keyword - search term - asin
    def Branded_NotBranded(row):
        """This helped me identify the weight of spending as well as our sales for branded and non-branded targeting"""
        keyword = str(row['Keyword_category_asin']).lower()  # Convert to string and lower case
        search_term = str(row['Customer Search Term']).lower()  # Convert to string and lower case
        brand_description = str(row['Brand Description']).lower()
        
        # List of example brands in lowercase
        brands = ['juhayna', 'molto', 'rameda', 'masrawy', 'biscomisr', 'seoudi', 'zahran', 'baraka', 'banque misr', 'cleopatra', 'edfa3ly', 'el araby', 'fresh', 'goldi', 'mezza luna', "temmy's"]

        # checking if our brand is mentioned either in search term or in targeting, this helps with deeper investigation to check our performance branded-nonbranded
        
        if any(brand in keyword for brand in brands):
                return 'Branded'
        
        if any(brand in search_term for brand in brands):
                return "Branded search term"

        # check if ASIN product isn't null, meaning that the customer landed on our product
        if  any(brand in brand_description for brand in brands):
                return 'Branded searched ASIN'

        return 'Not Branded'


    # Adding the branded_nonbranded column    
    cleaned_file['Branded_NotBranded'] = cleaned_file.apply(Branded_NotBranded, axis=1)


    # sort by date
    cleaned_file.sort_values(by=['Date'], inplace=True)

    #fillna
    cleaned_file.fillna('', inplace=True)

    # We follow a naming system for each campaign and we have to write the brand we're advertising to
    # The following is to export the brand name.
    cleaned_file['Brand'] = cleaned_file['Campaign Name'].str.split('_').str[0]

    #day of week
    cleaned_file['Day of week'] = cleaned_file['Date'].dt.dayofweek
    # map the day of the week number to its name
    cleaned_file['Day of week'] = cleaned_file['Day of week'].map({
        0: 'Monday',
        1: 'Tuesday',
        2: 'Wednesday',
        3: 'Thursday',
        4: 'Friday',
        5: 'Saturday',
        6: 'Sunday'
    })

    #Re-arranging columns and exporting only important columns
    cleaned_file = cleaned_file.reindex(columns=['Date',
    'Day of week',
    'Campaign Name',
    'Ad Group Name',
    'Brand',
    'Targeting Type',
    'Keyword_category_asin',
    'Branded_NotBranded',
    'Match Type',
    'Customer Search Term',
    'Impressions',
    'Clicks',
    'Spend',
    'Total Sales',
    'Total Orders',
    'Total Units',
    'Total Units Advertised ASINS',
    'Total Sales Advertised ASINS',    
    'Total Units Brand Halo ASIN',
    'Total Sales Brand Halo ASIN',
    'source_file_number'])


    # After cleaning the new dataset (most recent 90 days), we concatenate it with the main dataset(for entire ad account history)
    # remove any duplication so only the most updated results remain

    combined_df = pd.concat([cleaned_file,SPASIN_main_file])
    combined_df.sort_values(by=['source_file_number','Date','Campaign Name','Ad Group Name','Keyword_category_asin','Match Type','Customer Search Term'], ascending=[False,True,True,True,True,True,True], inplace=True)
    #remove duplicated
    
    combined_df.drop_duplicates(subset=['Date','Campaign Name','Ad Group Name','Keyword_category_asin','Match Type','Customer Search Term'], keep='first', inplace=True)


    return combined_df

# it's more efficient to export in csv.

Sponsored_Product_SPST(SPST,SPASIN_main_file).to_csv(r"C:\Work\\Unilever\ECOM\Base Reports\Final automated reports\Sponsored_Product_SPST_Unilever.csv", index=False)
